In [1]:
!apt-get update && apt-get install -y nvidia-cuda-toolkit
!nvidia-smi

!CMAKE_ARGS="-DGGML_CUDA=on" pip install https://github.com/abetlen/llama-cpp-python/releases/download/v0.3.4-cu122/llama_cpp_python-0.3.4-cp311-cp311-linux_x86_64.whl
!pip install huggingface_hub hf_transfer
%env HF_HUB_ENABLE_HF_TRANSFER=1
!huggingface-cli download bartowski/Mistral-Nemo-Instruct-2407-GGUF Mistral-Nemo-Instruct-2407-Q5_K_M.gguf --local-dir . --local-dir-use-symlinks False

!pip install --upgrade pip
!pip install --upgrade transformers sentencepiece datasets[audio]
!pip install torchsde
!pip install -U diffusers


from llama_cpp import Llama

# Loading the model multiple times can cause a crash

llm = Llama(
    model_path="./Mistral-Nemo-Instruct-2407-Q5_K_M.gguf",
    n_ctx=2048,
    n_gpu_layers=35,
    seed=1234

)
llm.verbose = False


from transformers import pipeline
from datasets import load_dataset
import soundfile as sf
import torch

from IPython.display import display
from IPython.display import Audio

synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts")

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    yes
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
llama_load_model_from_file: using device CUDA0 (Tesla T4) - 14992 MiB free
llama_model_loader: loaded meta data with 44 key-value pairs and 363 tensors from ./Mistral-Nemo-Instruct-2407-Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Mistral Nemo Instruct 2407
llama_model_loader: - kv   3:                            general.version str              = 2407
llama_model_loader: - kv   4:                           general.finetune str         

In [9]:
prompt = "tell me a joke"
prompt_template = f"[INST]Provide the information in clear, concise manner. {prompt}[/INST]"

response_list = []

for chunk in llm(
    prompt_template,
    max_tokens= 200, # speecht5_tts model has a maximum sequence length of 600 tokens
    stream= True,
    stop=["</s>"],
    echo=False,
    temperature=0.3

):
   response_list.append(chunk["choices"][0]["text"])

response = "".join(response_list)
print(response)


speech = synthesiser(response, forward_params={"speaker_embeddings": speaker_embedding})

sf.write("speech.wav", speech["audio"], samplerate=speech["sampling_rate"])

audio = Audio("speech.wav")
display(audio)

What do you call a fake noodle? An impasta.
